# Wrist decoder evaluation

In [ ]:
import os
import torch
from pytorch_lightning import Trainer
from hydra import initialize, compose
from hydra.utils import instantiate
from omegaconf import OmegaConf
from matplotlib import pyplot as plt
import seaborn as sns

from generic_neuromotor_interface.data import make_dataset

TASK_NAME = "wrist"
EMG_DATA_DIR = "~/emg_data"  # path to EMG data
MODELS_DIR = "~/emg_models"  # path to model files

## Download Data Subset and Model Files

Before running this notebook, make sure you have downloaded the data (small subset) and model checkpoint.

You can do so with the following cell:

In [ ]:
# Equivalent to README instructions.

from generic_neuromotor_interface.scripts.download_data import download_data
from generic_neuromotor_interface.scripts.download_models import download_models

download_data(TASK_NAME, "small_subset", EMG_DATA_DIR)
download_models(TASK_NAME, MODELS_DIR)

In [ ]:
if not os.path.exists(os.path.expanduser(EMG_DATA_DIR)):
    raise FileNotFoundError(f"The EMG data path does not exist: {EMG_DATA_DIR}")

if not os.path.exists(os.path.expanduser(MODELS_DIR)):
    raise FileNotFoundError(f"The models path does not exist: {MODELS_DIR}")

## Load model config

In [ ]:
"""Load model config"""

config_path = os.path.join(os.path.expanduser(MODELS_DIR), TASK_NAME, "model_config.yaml")
config = OmegaConf.load(config_path)

## Load model checkpoint

In [ ]:
"""Load model checkpoint"""

model_ckpt_path = os.path.join(
    os.path.expanduser(MODELS_DIR),
    TASK_NAME,
    "model_checkpoint.ckpt"
)
model = instantiate(config.lightning_module)
model = model.load_from_checkpoint(
    model_ckpt_path,
    map_location=torch.device("cpu"),
)

## Instantiate data module

In [ ]:
"""Assemble the data module"""

# Update DataModule config with data path
config["data_module"]["data_location"] = os.path.expanduser(EMG_DATA_DIR)
if "from_csv" in config["data_module"]["data_split"]["_target_"]:
    config["data_module"]["data_split"]["csv_filename"] = os.path.join(
        os.path.expanduser(EMG_DATA_DIR),
        f"{TASK_NAME}_corpus.csv"
    )

datamodule = instantiate(config["data_module"])

## Run inference on one test stage

In [ ]:
"""Grab one test stage"""

test_dataset = make_dataset(
    datamodule.data_location,
    partition_dict={
        "wrist_user_002_dataset_000": [(1713966045.6641605, 1713966207.9896057)]  # from wrist_mini_split.yaml
    },
    transform=datamodule.transform,
    emg_augmentation=None,
    window_length=None,
    stride=None,
    jitter=False,
)

sample = test_dataset[0]

In [ ]:
"""Run inference"""

EMG_SAMPLING_RATE = 2000.

model_output_sampling_rate = EMG_SAMPLING_RATE / model.network.stride

model.eval()

# unpack sample
emg = sample["emg"]
wrist_angles = sample["wrist_angles"]

# compute model outputs
with torch.no_grad():
    predictions = model(emg.unsqueeze(0))

predictions = predictions[0]

# convert wrist angle units from radians to degrees
wrist_angles = torch.rad2deg(wrist_angles)
predictions = torch.rad2deg(predictions)

# convert model predictions from displacements (deg) to velocities (deg/sec)
predictions *= model_output_sampling_rate

# estimate wrist angle velocities with the displacement across
# adjacent wrist angle measurements
wrist_angles_sliced = wrist_angles[:, model.network.left_context :: model.network.stride]  # slice the wrist angles at the model output sampling rate
wrist_angle_displacements = torch.diff(wrist_angles_sliced, dim=1)
wrist_angle_velocities = wrist_angle_displacements * model_output_sampling_rate  # convert to deg/sec

# since we don't know the true displacement at the first timestep, remove the first prediction
predictions = predictions[:, 1:]

In [ ]:
"""Evaluate Mean Absolute Error (MAE)"""

mae = torch.mean(torch.abs(predictions - wrist_angle_velocities))

print(f"MAE on this stage: {mae:.3f} (deg/sec)")

In [ ]:
"""Plot predictions and targets"""

fig, axes = plt.subplots(3, 1, figsize=(12, 7.5), sharex=True, sharey=False)

# plot EMG
ax = axes[0]
spacing = 120
for channel_index, channel_data in enumerate(emg):
    ax.plot(
        torch.arange(len(channel_data)) / EMG_SAMPLING_RATE,
        channel_data + channel_index * spacing,
        linewidth=1,
        color="0.7",
    )
ax.set_ylim([-spacing, len(emg) * spacing])
ax.set_yticks([])

sns.despine(ax=ax, left=True)

# wrist angles
ax = axes[1]
assert wrist_angles.shape[0] == 1
ax.plot(
    torch.arange(wrist_angles.shape[1]) / EMG_SAMPLING_RATE,
    wrist_angles[0],
    linewidth=1,
    color="k",
)
sns.despine(ax=ax)

# wrist angle velocities
ax = axes[2]
assert wrist_angle_velocities.shape[0] == 1
ax.plot(
    torch.arange(wrist_angle_velocities.shape[1]) / model_output_sampling_rate,
    wrist_angle_velocities[0],
    linewidth=1,
    color="k",
    label="ground truth",
)
sns.despine(ax=ax)

# model predictions
assert predictions.shape[0] == 1
ax.plot(
    torch.arange(predictions.shape[1]) / model_output_sampling_rate,
    predictions[0],
    linewidth=2,
    color="r",
    label="model predictions",
)

ax.legend(
    loc="upper left",
    ncols=1,
    bbox_to_anchor=(1.0, 1.0),
    frameon=False
)

ax.set_ylim([-300, 300])
ax.set_xlim([90, 110])

axes[0].set_ylabel("EMG\n(normalized)")
axes[1].set_ylabel("wrist angle\n(deg)")
axes[2].set_ylabel("wrist angle velocity\n(deg / sec)")
axes[2].set_xlabel("time\n(sec)");

## Evaluate full test set

Note that this requires you to have downloaded the full dataset (uncomment the below lines).

In [ ]:
## Uncomment to download the full dataset

# from generic_neuromotor_interface.scripts.download_data import download_data
# download_data(TASK_NAME, "full_data", EMG_DATA_DIR)

In [ ]:
trainer = Trainer(accelerator="cpu")
test_results = trainer.test(model=model, datamodule=datamodule)